![@mikegchambers](../images/header.png)

# Image Classification with SageMaker built-in algorithm

In this notebook, we use the SageMaker SDK to train an image classification model from an Amazon SageMaker built-in algorithm.

## Import Libraries

In [1]:
import sagemaker
import json 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

## Setup the SageMaker session 

The SageMaker SDK has some convenience methods for getting a reference to a role, setting up a session, and getting the location of a 'default' bucket that can be used.

In [2]:
role = sagemaker.get_execution_role()
sess = sagemaker.Session()
bucket=sess.default_bucket()

## Locate Data

When the SageMaker container launches, we will pass in the location of the training data.  The data is expected to be in an S3 bucket.  We set this here such that we can use it later.  The location of this data is in a publicly accessible bucket in one of my accounts.  

We also set the location where we will save the completed model.

In [10]:
s3train = 's3://aws-mls-c01/cifar10/train/cifar10_train.rec'
s3validation = 's3://aws-mls-c01/cifar10/validation/cifar10_val.rec'

s3_output_location = 's3://{}/image-classificaiton/output'.format(bucket)

## Define the training image

Here we point SageMaker to the container we want to use.  In this case, the built in 'image-classification' algorithm/container is being used.

In [11]:
training_image = sagemaker.image_uris.retrieve('image-classification', region='us-east-1')

## Create a SageMaker Estimator

The SageMaker Estimator is one of the key object types in the SageMaker SDK.  Here we initialise the estimator specifying the instance type, how many instances we want to use, and other parameters including the use of spot instances. 

This is how SageMaker manages infrastructure for us with a simple SDK call to the API.

In [12]:
ic = sagemaker.estimator.Estimator( training_image,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.p3.16xlarge',
                                    volume_size = 50,
                                    max_run = 7200,
                                    input_mode= 'File',
                                    output_path=s3_output_location,
                                    sagemaker_session=sess,
                                    use_spot_instances=True,
                                    max_wait=7200)

Then we set some hyperparameters:

In [13]:
ic.set_hyperparameters(             use_pretrained_model=1,
                                    num_layers=50,
                                    image_shape = "3,32,32",
                                    num_classes=10,
                                    num_training_samples=50000,
                                    mini_batch_size=64,
                                    epochs=5,
                                    learning_rate=0.001,
                                    optimizer='adam')

Define our input channels:

In [14]:
train_data = sagemaker.inputs.TrainingInput(     s3train, 
                                                 distribution='FullyReplicated', 
                                                 content_type='application/x-recordio', 
                                                 s3_data_type='S3Prefix')

validation_data = sagemaker.inputs.TrainingInput(s3validation, 
                                                 distribution='FullyReplicated', 
                                                 content_type='application/x-recordio', 
                                                 s3_data_type='S3Prefix')

data_channels = {'train': train_data, 'validation': validation_data}

## Train the model

And finally, we call `fit` to train the model.

In [15]:
ic.fit(inputs=data_channels)

2021-03-04 11:08:42 Starting - Starting the training job...
2021-03-04 11:09:06 Starting - Launching requested ML instancesProfilerReport-1614856121: InProgress
.........
2021-03-04 11:10:29 Starting - Preparing the instances for training......
2021-03-04 11:11:37 Downloading - Downloading input data...
2021-03-04 11:12:08 Training - Downloading the training image......
2021-03-04 11:13:11 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
[03/04/2021 11:13:15 INFO 140410731967616] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/image_classification/default-input.json: {u'beta_1': 0.9, u'gamma': 0.9, u'beta_2': 0.999, u'optimizer': u'sgd', u'use_pretrained_model': 0, u'eps': 1e-08, u'epochs': 30, u'lr_scheduler_factor': 0.1, u'num_layers': 152, u'image_shape': u'3,224,224', u'precision_dtype': u'float32', u'mini_batch_size': 32, u'weight_decay': 0.0001, u'learning_rate': 0.1, u'momentum': 0}
[0

[03/04/2021 11:14:03 INFO 140410731967616] Epoch[0] Batch [440]#011Speed: 1082.602 samples/sec#011accuracy=0.299922
[03/04/2021 11:14:04 INFO 140410731967616] Epoch[0] Batch [460]#011Speed: 1088.182 samples/sec#011accuracy=0.299349
[03/04/2021 11:14:05 INFO 140410731967616] Epoch[0] Batch [480]#011Speed: 1093.924 samples/sec#011accuracy=0.298174
[03/04/2021 11:14:06 INFO 140410731967616] Epoch[0] Batch [500]#011Speed: 1098.015 samples/sec#011accuracy=0.295939
[03/04/2021 11:14:07 INFO 140410731967616] Epoch[0] Batch [520]#011Speed: 1101.892 samples/sec#011accuracy=0.292496
[03/04/2021 11:14:08 INFO 140410731967616] Epoch[0] Batch [540]#011Speed: 1106.406 samples/sec#011accuracy=0.290521
[03/04/2021 11:14:09 INFO 140410731967616] Epoch[0] Batch [560]#011Speed: 1111.078 samples/sec#011accuracy=0.288269
[03/04/2021 11:14:10 INFO 140410731967616] Epoch[0] Batch [580]#011Speed: 1115.272 samples/sec#011accuracy=0.287220
[03/04/2021 11:14:11 INFO 140410731967616] Epoch[0] Batch [600]#011Speed

[03/04/2021 11:15:12 INFO 140410731967616] Epoch[2] Batch [140]#011Speed: 1228.098 samples/sec#011accuracy=0.335882
[03/04/2021 11:15:13 INFO 140410731967616] Epoch[2] Batch [160]#011Speed: 1226.922 samples/sec#011accuracy=0.335986
[03/04/2021 11:15:14 INFO 140410731967616] Epoch[2] Batch [180]#011Speed: 1225.460 samples/sec#011accuracy=0.336067
[03/04/2021 11:15:16 INFO 140410731967616] Epoch[2] Batch [200]#011Speed: 1225.682 samples/sec#011accuracy=0.334499
[03/04/2021 11:15:17 INFO 140410731967616] Epoch[2] Batch [220]#011Speed: 1225.992 samples/sec#011accuracy=0.333215
[03/04/2021 11:15:18 INFO 140410731967616] Epoch[2] Batch [240]#011Speed: 1227.650 samples/sec#011accuracy=0.330978
[03/04/2021 11:15:19 INFO 140410731967616] Epoch[2] Batch [260]#011Speed: 1228.997 samples/sec#011accuracy=0.328424
[03/04/2021 11:15:20 INFO 140410731967616] Epoch[2] Batch [280]#011Speed: 1228.763 samples/sec#011accuracy=0.327569
[03/04/2021 11:15:21 INFO 140410731967616] Epoch[2] Batch [300]#011Speed

[03/04/2021 11:16:22 INFO 140410731967616] Epoch[3] Batch [660]#011Speed: 1232.187 samples/sec#011accuracy=0.358288
[03/04/2021 11:16:23 INFO 140410731967616] Epoch[3] Batch [680]#011Speed: 1231.878 samples/sec#011accuracy=0.360109
[03/04/2021 11:16:24 INFO 140410731967616] Epoch[3] Batch [700]#011Speed: 1231.757 samples/sec#011accuracy=0.361938
[03/04/2021 11:16:25 INFO 140410731967616] Epoch[3] Batch [720]#011Speed: 1231.769 samples/sec#011accuracy=0.363146
[03/04/2021 11:16:27 INFO 140410731967616] Epoch[3] Batch [740]#011Speed: 1231.243 samples/sec#011accuracy=0.364879
[03/04/2021 11:16:28 INFO 140410731967616] Epoch[3] Batch [760]#011Speed: 1231.088 samples/sec#011accuracy=0.366171
[03/04/2021 11:16:29 INFO 140410731967616] Epoch[3] Batch [780]#011Speed: 1230.650 samples/sec#011accuracy=0.367338
[03/04/2021 11:16:29 INFO 140410731967616] Epoch[3] Train-accuracy=0.367338
[03/04/2021 11:16:29 INFO 140410731967616] Epoch[3] Time cost=40.564
[03/04/2021 11:16:30 INFO 140410731967616] 

## Create an inference endpoint

Now that the model is created (and saved to S3 at `s3_output_location`) we can create an endpoint from the model, such that we can use it to make inference about new data.

In [16]:
ic_classifier = ic.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

-----------------!

Within the SageMaker SDK, we don't need to know the endpoint name, as we can simply reference it from the classifier we just made.  But it might be useful to know, so let's find out: 

In [ ]:
ic_classifier.endpoint_name

Now let's create some data to use for testing.  Here we set the labels, and a test image:

In [ ]:
labels = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

f = open('./test-images/ship.jpg', 'rb') # opening a binary file
data = f.read()

## Make an inference/prediction

With our endpoint deployed and sample data ready, we can call `predict` and see what we find.

As this model was trained quickly, don't expect anything too amazing!  If you want to improve the accuracy change some of the hyperparameters and train again.  The first thing to try is to increase the number of epochs.

In [ ]:
prediction = ic_classifier.predict(data, initial_args={"ContentType": "application/x-image"})
probs = json.loads(prediction)

print(probs)

And make it pretty:

In [ ]:
figure(num=None, figsize=(8, 5), dpi=80, facecolor='w', edgecolor='k')
plt.bar(range(10), probs)
plt.xticks(range(10), labels)
plt.show()

In [ ]:
index_of_prediction = np.argmax(probs)
label_of_prediciton = labels[index_of_prediction]

print("This looks like a {}.".format(label_of_prediciton))

## Clean up

Now let's tear down the endpoint as we are charged whilst it's up and running. 

In [ ]:
ic_classifier.delete_endpoint()